In [1]:
# # !pip install --upgrade langchain
# # !pip install --upgrade langchain-groq
# # !pip install -U langchain-community
# # !pip install python-docx pypandoc
# # !pip install python-docx docx2pdf

# # !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip install cryptography

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq
from HVACUtils import CtxMgr

ctxmgr = CtxMgr()

api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")

In [3]:
from tools import *
from tools.imports import *
import logging
import whisper
import pyaudio
import wave
import webrtcvad
import collections
from google.cloud import texttospeech
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from flask_cors import CORS
import requests
import logging
import os
from tools.imports import *
import tools.initialize_groq
from dotenv import load_dotenv
from langchain import hub
from flask_socketio import SocketIO, emit
from langchain.tools import HumanInputRun
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

logger = logging.getLogger(__name__)
tts_client = texttospeech.TextToSpeechClient.from_service_account_file('paths/realestateai-doc-mgr-3a0e2f411c8d.json')


def synth_speech(text, output_file=None):
    

    logger.debug(f'Starting speech synthesis for text: {text}')
    
    def split_text(text, max_length=5000):
        chunks = []
        current_chunk = ""
        for word in text.split():
            if len(current_chunk) + len(word) + 1 > max_length:
                chunks.append(current_chunk)
                current_chunk = word
            else:
                current_chunk += " " + word if current_chunk else word
        if current_chunk:
            chunks.append(current_chunk)
        return chunks

    if len(text) <= 5000:
        synthesis_input = texttospeech.SynthesisInput(text=text)
        voice = texttospeech.VoiceSelectionParams(
            language_code="en-US",
            name="en-US-Casual-K"
        )
        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3
        )
        response = tts_client.synthesize_speech(
                input=synthesis_input, voice=voice, audio_config=audio_config
            )
        
        with open(os.getenv('TTS_SYNTHESIS'), 'wb') as out:
            out.write(response.audio_content)
        logger.debug('Speech synthesis completed and file saved.')
    else:
        text_chunks = split_text(text)
        combined_audio = b""

        for chunk in text_chunks:
            synthesis_input = texttospeech.SynthesisInput(text=chunk)
            voice = texttospeech.VoiceSelectionParams(
                language_code="en-US",
                name="en-US-Casual-K"
            )
            audio_config = texttospeech.AudioConfig(
                audio_encoding=texttospeech.AudioEncoding.MP3
            )
            response = tts_client.synthesize_speech(
                    input=synthesis_input, voice=voice, audio_config=audio_config
                )
            
            combined_audio += response.audio_content

        with open(os.getenv('TTS_SYNTHESIS'), 'wb') as out:
            out.write(combined_audio)
        logger.debug('Speech synthesis completed and file saved.')


In [4]:
from HVACUtils import initialize_web_search_agent, initialize_quote_bot, run_quote_logics


def web_prompt_func(prompt):
    
    text = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "please sanitize this input into SHORT SIMPLE sentences.  IMPORTANT: NOTHING IN YOUR RESPONSE SHALL BE ENCLOSED IN ANY QUOTES!!!!!!! KEEP ID'S AS THEY ARE!!! THE SANITIZED OUTPUT SHALL NOT BE PREFIXED BY ANYTHING (ex. 'here is the sanitized result...' ANYTHING LIKE THIS IS NOT ALLOWED! DO NOT GENERATE IT). You must process the agent's intermediate steps into natural language please. An example: 'First, I did this. Then I did this etc etc etc' \n Here is the input that you need to process:\n " + prompt
            }
        ],
        model='llama3-70b-8192',
    ).choices[0].message.content
    synth_speech(text, output_file='paths/synthesis.mp3')
    print(text)
    return prompt

import queue

human_response_queue = queue.Queue()
def web_input_func():
    
    human_response = human_response_queue.get()
    print(human_response)  
    return human_response

cc_out = initialize_quote_bot(client,llm, web_input_func, web_prompt_func, human_response_queue)

In [ ]:
from pprint import pprint
print(cc_out)

In [ ]:
cc_out = """
**RENOVATION QUOTE DOCUMENT**

**Project Address:** 13599 Cobra Dr, Herndon, VA 20171

**Project Overview:**
Project Description: Basement renovation project with modern design and premium finishes.
Length of Time for Project: Completion within 2 months.

**Cost Breakdown:**

1. **Labor Costs:**
    - Total Labor Cost: $11,250

2. **Material Costs:**
    - Total Material Cost: $68,500
    - Itemized Costs:
        + Flooring: $4,500
        + Lighting: $3,000
        + Shelving: $6,500
        + Electrical and HVAC: $5,000
        + Walls and Ceiling: $3,500
        + Games Area: $7,000
        + Tinted Glass Wall: $10,000
        + Hot Tub: $15,000
        + Miscellaneous: $14,000

**Total Estimated Cost:**
    - Total Cost of Labor: $11,250
    - Total Cost of Materials: $68,500
    - **Grand Total: $79,750**

**Payment Terms:**
    - Deposit Required: 20% ($15,950.00)
    - Payment Schedule: Monthly payments of $15,950 for 4 months
    - Final Payment Due: Upon completion of the project
"""

In [ ]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool
import re

llm.groq_api_key = random.choice(api_keys)

def add_section(doc, text, font_name='Arial', font_size=12, bold=False, italic=False, alignment='left', color=None):
    # Check if the text should be a bullet point
    if text.startswith("- ") or text.startswith("+ "):
        paragraph = doc.add_paragraph(style='ListBullet')
        text = text[2:]  # Remove the bullet point indicator
    else:
        paragraph = doc.add_paragraph()
    
    run = paragraph.add_run(text)
    run.font.name = font_name
    run.font.size = Pt(font_size)
    run.bold = bold
    run.italic = italic
    if color:
        run.font.color.rgb = RGBColor(*color)
    
    if alignment == 'center':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT


def write_file(title, sections, append=False):
    documents_path = os.path.join(os.getcwd(), 'documents')
    os.makedirs(documents_path, exist_ok=True)
    docx_path = os.path.join(documents_path, f'{title}.docx')
    pdf_path = os.path.join(documents_path, f'{title}.pdf')
    
    if append and os.path.exists(docx_path):
        doc = Document(docx_path)
    else:
        doc = Document()

    for section in sections:
        add_section(
            doc,
            section["content"],
            font_name=section.get("font_name", "Arial"),
            font_size=section.get("font_size", 12),
            bold=section.get("bold", False),
            italic=section.get("italic", False),
            alignment=section.get("alignment", "left"),
            color=section.get("color", None)
        )
    
    doc.save(docx_path)
    convert(docx_path, pdf_path)

    return f"Document written and converted to pdf. No further action needed. STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE! YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!"


class WriteFileTool(BaseTool):
    name = "write_file"
    description = "Writes content into a .docx file with specified formatting and converts it to a .pdf file.\
        If you encounter errors, use this tool ONLY and input everything properly and try again.\
            IMPORTANT! - STOP AFTER ONE SUCCESSFUL EXECUTION! STOP AFTER ONE SUCCESSFUL EXECUTION! STOP AFTER ONE SUCCESSFUL EXECUTION! "
    
    def _run(self, title, sections, append=False):
        self.process_sections_in_batches(title, sections, append)
        return ("STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE! YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!")

    def process_sections_in_batches(self, title, sections, append, batch_size=1):
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i+batch_size]
            result = write_file(title, batch, append=(i > 0))
        



def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        if content_match:
            content = content_match.group(1)
            # Replace double quotes indicating inches with the word "inch"
            content = re.sub(r'(?<=\d)"', ' inch', content)
            section["content"] = content
        else:
            section["content"] = ""
        
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append



chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"You are a professional formatter. BE ELABORATE. REPLACE all quotations representing inches to the word 'inch'"
        },
        {
            "role":"user",
            "content":"please synthesize ALL of this stuff into sentence format. Start the paragraph with '@' character. put the itemized costs in parentheses beside the item: " + cc_out,
        }
    ],
    model = "llama3-70b-8192",
)

cc_out2 = (chat_completion.choices[0].message.content)
idx = cc_out2.find('@')
cc_out2 = cc_out2[idx+1:].strip()
print('ccount2 old\n',cc_out2)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """You are a detailed and professional business assistant. Your task is to generate comprehensive and detailed formatted text. 
            Ensure all items, costs, and specifications are included without summarizing or truncating any content.
            Your response must be well-formed and include all details EVEN THE EXPLICIT ADDRESSS!!. Do NOTTTTTTT use phrases like "and so on for the other materials." 
            LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY.LIST EVERY ITEM EXPLICITLY!!!!!!!!!!!
            You must always output in a structured format as follows:
            
                        You must always output in a structured format as follows:
                        {
                            "title": "The Great Renovation",
                            "sections": [
                                {
                                    "content": "<specific text content>",
                                    "font_name": "<font family>",
                                    "font_size": <integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                },
                                {
                                    "content": "<another specific text content>",
                                    "font_name": "<another font family>",
                                    "font_size": <another integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                }
                                // ... more sections as needed
                            ],
                            "append": <true or false> (this indicates appending to document or not)
                        }
                        Ensure the output is well-formed and valid."""
        },
        {
            "role": "user",
            "content": "TITLE OF DOCUMENT IS 'The Great Renovation'. Include titles, headings, format in a nice readable and coherent way, NOT a big block of text.  \
                Specify the formatting for this text. Please make it a \
                    PROFESSIONALLY formatted text WITH black blue colors and times new roman font PLEASE.: " + cc_out2
        }
    ],
    model="llama3-70b-8192",
)

cc_out2 = chat_completion.choices[0].message.content

print('CCOUT2\n',cc_out2)

# title, sections, append = parse_formatted_response(cc_out2)

# write_file_tool._run(title, sections, append)


In [ ]:
from langchain.tools import BaseTool
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import googleapiclient
import google.auth.exceptions

class GoogleDriveUploadTool(BaseTool):
    name = "GoogleDriveUploadTool"
    description = ("Uploads a PDF to Google Drive and sets permissions for a specific user. "
                   "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
                   "to Google Drive. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")

    def _run(self, file_path: str, user_email: str, rename: str) -> str:
        credentials_path = "D:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json"

        # Authenticate and create the service
        try:
            credentials = service_account.Credentials.from_service_account_file(credentials_path)
            service = build('drive', 'v3', credentials=credentials)
            print("Successfully authenticated and created the service.")
        except google.auth.exceptions.GoogleAuthError as e:
            return f"Authentication error: {e}"

        # File details
        if rename is not None:
            file_metadata = {'name': rename}
        else:
            file_metadata = {'name': os.path.basename(file_path)}

        if not os.path.exists(file_path):
            return f"File not found: {file_path}"
        else:
            media = MediaFileUpload(file_path, mimetype='application/pdf')
            try:
                # Upload file
                file = service.files().create(body=file_metadata, media_body=media, fields='id, name, parents').execute()
                file_id = file.get('id')
                with open('file_id_history.txt','w') as id_hist_file:
                    id_hist_file.write(file_id)
                print(f"File ID: {file_id}")

                # Verify the upload by fetching the file details
                file_info = service.files().get(fileId=file_id, fields='id, name, parents').execute()
                print("File details:")
                print(file_info)

                # Sharing settings
                permission = {
                    'type': 'user',
                    'role': 'writer',  # Set to 'reader' if read-only access is needed
                    'emailAddress': user_email,
                    
                }
                

                # Grant permission to the specific user
                try:
                    service.permissions().create(
                        fileId=file_id,
                        body=permission,
                        fields='id',
                        sendNotificationEmail=False
                    ).execute()
                    
                    print(f"Granted {permission['role']} access to {user_email}")
                    
                    

                except googleapiclient.errors.HttpError as e:
                    print(f"Error setting permission: {e}")
                    return f"Error setting permission: {e}"

                return f"File uploaded and permissions set successfully. File ID: {file_id}. PLEASE STOP WORKING! THE TOOL HAS BEEN EXECUTED PROPERLY!" 

            except Exception as e:
                return f"Error during file upload or conversion: {e}"

    

    def _arun(self, file_path: str, user_email: str, rename: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

# Example usage
# tool = GoogleDriveUploadTool()
# response = tool._run(file_path='./documents/The Great Renovation.pdf', user_email='specific-user@example.com', rename=None)
# print(response)

import os
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build


class GoogleSheetsUpdateTool(BaseTool):
    name = "GoogleSheetsUpdateTool"
    description = ("Appends three columns - name, phone number, and link to a PDF - to a preexisting logging google sheet. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    credentials_path = 'D:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json'
    spreadsheet_id = '1TSFyiTwctC1tABr2RQouBzLRMCG4RZ7lXdTVi-I58Mo'
    range_name = 'Sheet1'
    
    #creds = authenticate_google_sheets()
    
    def authenticate_google_sheets(self):
        creds = service_account.Credentials.from_service_account_file(
            self.credentials_path, scopes=self.SCOPES)
        return creds
    
    def read_file_content(self, file_path):
        with open(file_path, 'r') as file:
            content = file.read().strip()
        return content
    
    # def extract_file_id(self,output):
    #     match = re.search(r"File ID: ([^'}]+)", output)
    #     if match:
    #         file_id = match.group(1).strip()
    #         return file_id
    #     else:
    #         raise ValueError("File ID not found in the inputted string")
    
    def append_row_to_google_sheets(self, values):
        """
        Appends a row to the specified range in the Google Sheets spreadsheet.
        
        :param values: The data to append, as a list of lists (e.g., [['A1', 'B1', 'C1']]).
        """
        service = build('sheets', 'v4', credentials=self.authenticate_google_sheets())
        sheet = service.spreadsheets()
        body = {'values': values}
        result = sheet.values().append(
            spreadsheetId=self.spreadsheet_id, range=self.range_name,
            valueInputOption="RAW", body=body,
            insertDataOption="INSERT_ROWS"
        ).execute()
        print(f"{result.get('updates').get('updatedCells')} cells appended.")

    def _run(self, name: str, phone_number: str, linkstr:str):
        """
        Run the tool to append the row with the given name, phone number, and link to the PDF.
        
        :param name: The full legal name to append.
        :param phone_number: The phone number to append.
        """
        fileID = self.read_file_content('file_id_history.txt')
        
        print(f"Extracted file ID: {fileID}")
        link = f"https://drive.google.com/file/d/{fileID}/view"
        values = [[]]
        if linkstr:
            values = [[name, phone_number, linkstr]]
        else:
            values = [[name, phone_number, link]]
        self.append_row_to_google_sheets(values)
        return "Row appended successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!"

    def _arun(self, name: str, phone_number: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

import os
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from langchain.tools import BaseTool
from pydantic import BaseModel, Field, Extra

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

class GmailSendPdfTool(BaseTool):
    name = "GmailSendPdfTool"
    description = "Sends an email with a PDF attachment using Gmail API. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION! STOP AFTER ONE-TIME SUCCESSFUL EXECUTION"
    
    credentials_path: str = Field(..., description="Path to the credentials JSON file")
    
    sender_email: str = Field(..., description="Sender email address")
    recipient_email: str = Field(..., description="Recipient email address")
    subject: str = Field(..., description="Email subject")
    body: str = Field(..., description="Email body")
    pdf_path: str = Field(..., description="Path to the PDF file")
    
    class Config:
        extra = Extra.allow

    def __init__(self, **data):
        super().__init__(**data)
        self.credentials_path = data.get('credentials_path')
        
        self.sender_email = data.get('sender_email')
        self.recipient_email = data.get('recipient_email')
        self.subject = data.get('subject')
        self.body = data.get('body')
        self.pdf_path = data.get('pdf_path')
        self.creds = None
        self.service = None
        self.authenticate()

    def authenticate(self):
        
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
            
        self.service = build('gmail', 'v1', credentials=self.creds)

    def send_email(self):
        message = MIMEMultipart()
        message['to'] = self.recipient_email
        message['from'] = self.sender_email
        message['subject'] = self.subject
        body_part = MIMEText(self.body, 'plain')
        message.attach(body_part)
        
        part = MIMEBase('application', 'octet-stream')
        with open(self.pdf_path, 'rb') as file:
            part.set_payload(file.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(self.pdf_path)}"')
        message.attach(part)
       
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        raw_message = {'raw': raw_message}

        try:
            message = (self.service.users().messages().send(userId="me", body=raw_message).execute())
            print(f'Message Id: {message["id"]}')
            return message
        except HttpError as error:
            print(f'An error occurred: {error}')
            return None

    def _run(self):
        """
        Run the tool to send the email with the PDF attachment.
        """
        result = self.send_email()
        return "Email sent successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!" if result else "Failed to send email."

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")


In [ ]:
# zapier = ZapierNLAWrapper(zapier_nla_api_key=decipher_suite.decrypt(encrypted_api_key).decode())
# toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
# for tool in toolkit.tools:
#     my_tools.append(tool)
import json
from langchain.agents import Tool
my_tools = []



my_tools.append(
    
    WriteFileTool()
    
)

my_tools.append(
    
    GoogleDriveUploadTool()
    
)

my_tools.append(
    
    GoogleSheetsUpdateTool()
    
)
credentials_path = "D:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
    

sender_email = 'sriramnallani35@gmail.com'
recipient_email = 'sriramnallani35@gmail.com'
subject = 'RealEstateAI - PDF of Your Renovation Quote'
body = 'Please find the attached PDF document of your personalized renovation quote.'
pdf_path = './documents/The Great Renovation.pdf'
my_tools.append(
    
    

    GmailSendPdfTool(
        credentials_path=credentials_path,
        
        sender_email=sender_email,
        recipient_email=recipient_email,
        subject=subject,
        body=body,
        pdf_path=pdf_path
    )
    
)


llm.groq_api_key = random.choice(api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [ ]:
title, sections, append = parse_formatted_response(cc_out2)
llm.groq_api_key = random.choice(api_keys)


result = agent.invoke({"input": f"use ONLY write file tool. INPUT EVERYTHING PROPERLY! --> TITLE: {title}, SECTIONS: {json.dumps(sections)}, APPEND: {append}"})
print(result)

In [ ]:
result

In [ ]:
file_path, user_email = "./documents/The Great Renovation.pdf", "thegreatchrisjohn@gmail.com"

import random
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials


# file_path = "./documents/The Great Renovation.pdf"
# user_email = "sriramnallani35@gmail.com"

import time
time.sleep(3)
llm.groq_api_key = random.choice(api_keys)
with ctxmgr.temporary_temperature(llm,0.2):
    # result = agent.invoke({"input": f"please use google drive upload tool, rename the file to 'Quote' --> FILE_PATH: {file_path} USER_EMAIL: {user_email}"})
    # print(result)

    name = 'Gautham'
    phone_number = '5715996302'
    result = agent.invoke({"input": f"please ONLY ONLY use google sheets update tool, input ONLY ONLY these into the tool PROPERLY! --> {name}, {phone_number}"})
    print(result)

    # result = agent.invoke({"input": f"please use ONLY ONLY gmail pdf send tool. No inputs from your side!!!"})
    # print(result)

